In [95]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests
import copy 

# Lexique:

* ROME: Répertoire opérationnel des métiers et des emplois

---

---

In [96]:
# open ideo-formations_initiales_en_France.csv
df = pd.read_csv('ideo-formations_initiales_en_france.csv', sep = ";")
df.head()


,code CNIS,sigle type formation,libellé type formation,libellé formation principal,sigle formation,durée,code RNCP,niveau RNCP,libellé niveau RNCP,tutelle,lien site onisep.fr,domaine/sous-domaine
0,NaN,NaN,formation d'école spécialisée,Chargé d'accompagnement social et professionnel,NaN,2 ans,7426.0,3.0,niveau 5 (bac + 2),non renseigné,http://www.onisep.fr/http/redirection/formatio...,santé - social - sport/travail social
1,NaN,FCIL,formation complémentaire d'initiative locale,FCIL Art textile et spectacle vivant,NaN,1 an,NaN,0.0,non inscrit au RNCP,non renseigné,http://www.onisep.fr/http/redirection/formatio...,"arts, culture, artisanat/artisanat d'art| arts..."
2,NaN,NaN,formation d'école spécialisée,Manager social média et webmarketing,NaN,2 ans,NaN,0.0,non inscrit au RNCP,non renseigné,http://www.onisep.fr/http/redirection/formatio...,"information-communication, audiovisuel/communi..."
3,310.0,NaN,diplôme d'école de commerce visé de niveau bac...,Diplôme en économie financière et managériale ...,NaN,2 ans,NaN,1.0,niveau 7 (bac + 5),non renseigné,http://www.onisep.fr/http/redirection/formatio...,banque - assurances - immobilier/banque| banqu...
4,NaN,NaN,diplôme d'ingénieur,diplôme d'ingénieur de l'École nationale supér...,ENSIBS,3 ans,35798.0,1.0,niveau 7 (bac + 5),Ministère chargé de l'Enseignement supérieur d...,http://www.onisep.fr/http/redirection/formatio...,mécanique/automatismes| électricité - électron...


In [97]:
# tranform xlsx into csv
read_file = pd.read_excel (r'Libelles_metiers_numerique_collecte_2022.xlsx')
read_file.to_csv (r'Libelles_metiers_numerique_collecte_2022.csv', index = None, header=True)

In [98]:
# garder uniquement les colonnes liées aux formations du web (via Libelles_metiers_numerique_collecte_2022.csv ?)
df_libelles = pd.read_csv('Libelles_metiers_numerique_collecte_2022.csv')

# on enleve la colonne "libellé féminin car pas utile ici, rajoute de la complexité à notre projet
df_libelles = df_libelles.drop(columns=['Libellé féminin', 'Liste'])
df_libelles


,Libellé masculin,Numérique
0,Acheteur d'espaces publicitaires web,4
1,Acheteur média,4
2,Adjoint technicien d'exploitation informatique,1
3,Administrateur base de données,1
4,Administrateur bureautique,1
...,...,...
380,Web content manager,4
381,Web designer,4
382,Web marketeur,4
383,Webmaster,4


In [99]:
df_libelles.describe()

,Numérique
count,385.000000
mean,3.051948
std,1.771417
min,1.000000
25%,1.000000
50%,3.000000
75%,4.000000
max,6.000000


In [100]:
df_libelles = df_libelles.rename(columns={"Liste": "liste", "Numérique": "numerique"})

In [101]:
# Numerique correspond à la catégorie du métier parmis les 6 suivantes: 
# 1 = Les métiers des infrastructures réseaux, télécommunications ;
# 2 = Les métiers de l’informatique, des systèmes d’information ;
# 3 = Les métiers de l’analyse de données et de l’intelligence artificielle ;
# 4 = Les métiers de la communication, de l’interface utilisateur et de la création numérique ;
# 5 = Les métiers du management et de la stratégie ;
# 6 = Les métiers de l’expertise et du conseil.

# crete a new column 'category' with the category of the job depending on the value of 'numerique'
df_libelles['category'] = df_libelles['numerique'].apply(lambda x: 'Infrastructures & Télécommunication' if x == 1 else 'Reseaux' if x == 2 else 'Analyse' if x == 3 else 'Communication & Création' if x == 4 else 'Management' if x == 5 else 'Expertise & Conseil')

In [102]:
df_libelles

,Libellé masculin,numerique,category
0,Acheteur d'espaces publicitaires web,4,Communication & Création
1,Acheteur média,4,Communication & Création
2,Adjoint technicien d'exploitation informatique,1,Infrastructures & Télécommunication
3,Administrateur base de données,1,Infrastructures & Télécommunication
4,Administrateur bureautique,1,Infrastructures & Télécommunication
...,...,...,...
380,Web content manager,4,Communication & Création
381,Web designer,4,Communication & Création
382,Web marketeur,4,Communication & Création
383,Webmaster,4,Communication & Création


In [ ]:
# grâce aux catégories du csv libelles, on peut essayer de croiser les catégories aux codes de métiers du csv 

In [105]:
# tranform xlsx into csv
df_rncp = pd.read_xml(r'rncp-rs-france-competence-v2.0.xsd')
df_rncp.head()

,name,{http://www.w3.org/2001/XMLSchema}complexType,{http://www.w3.org/2001/XMLSchema}unique,{http://www.w3.org/2001/XMLSchema}restriction,{http://www.w3.org/2001/XMLSchema}simpleType
0,FICHES,NaN,NaN,NaN,NaN
1,siretType,NaN,NaN,NaN,NaN
2,numeroType,NaN,NaN,NaN,NaN
3,OuiNonType,NaN,NaN,NaN,NaN
4,FORMACODES,NaN,NaN,NaN,NaN
